# Инструменты функционального программирования

## Lection

** Variable Scope Rules **

In [25]:
def f1(a):
    print(a)
    print(b)
f1(3)
# b1 = 6

3


NameError: name 'b' is not defined

In [2]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [4]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)
print(b)

3
6
9


[to read about dis](https://docs.python.org/3/library/dis.html)

[to read about digits](https://stackoverflow.com/questions/12673074/how-should-i-understand-the-output-of-dis-dis)

In [5]:
from dis import dis

dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [6]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [7]:
dis(f3)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


** Closures **

A Closure is a function object that remembers values in enclosing scopes

In [12]:
def make_averager():
    data = []
    x=3
    def averager(new_value):
        data.append(new_value)
        total = sum(data)
        return total/len(data)
    return averager
avg = make_averager()
avg(10)
avg(15)

12.5

In [13]:
avg.__code__.co_varnames

('new_value', 'total')

In [14]:
avg.__code__.co_freevars

('data',)

In [15]:
avg.__closure__

(<cell at 0x000002582DFC71F0: list object at 0x000002582DC6EAC0>,)

In [19]:
avg.__closure__[0].cell_contents

[10, 15]

** non local **

In [21]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [22]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager
avg = make_averager()
avg(10)

10.0

In [26]:
print(avg.__closure__[1].cell_contents)
print(avg.__closure__[0].cell_contents)

10
1


** Decorators **

** 1 **

In [27]:
def get_text():
    return "this is the text"

def add_div(fn):
    def wrapper(): 
        return "<div>"+fn()+"</div>"
    return wrapper
print(get_text())

get_text = add_div(get_text)

print(get_text())

this is the text
<div>this is the text</div>


In [28]:
def add_div(fn):
    def wrapper(): 
        return "<div>"+fn()+"</div>"
    return wrapper
@add_div
def get_text():
    return "this is the text"
get_text()

'<div>this is the text</div>'

** 2 **

In [29]:
def add_div(fn):
    def wrapper(txt): 
        return "<div>"+fn(txt)+"</div>"
    return wrapper
@add_div
def get_text(txt):
    return txt
get_text("hello")

'<div>hello</div>'

** 3 **

In [30]:
def add_tag(tag):
    def add_div(fn):
        def wrapper(txt): 
            return f"<{tag}>{fn(txt)}</{tag}>"
        return wrapper
    return add_div
@add_tag('span')
def get_text(txt):
    return txt
get_text("hello")

'<span>hello</span>'

** 4 **

In [31]:
@add_tag('span')
@add_tag('div')
def get_text(txt):
    return txt
get_text("hello")

'<span><div>hello</div></span>'

** 5 **

In [19]:
def add_tag(fn = None, *, tag = "span" ):
    if fn is None:
        return lambda fn: add_tag(fn, tag = tag)
    def wrapper(txt): 
        return f"<{tag}>{fn(txt)}</{tag}>"
    return wrapper

@add_tag(tag = "span")
def get_text(txt):
    return txt
get_text("hello")

'<span>hello</span>'

In [1]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner
@trace
def identity(x):
    "I do nothing useful."
    return x
identity(42)

identity (42,) {}


42

In [2]:
help(identity)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [3]:
identity.__doc__

In [4]:
def identity(x):
    "I do nothing useful."
    return x


In [5]:
identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

In [6]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner
@trace
def identity(x):
    "I do nothing useful."
    return x

In [7]:
identity.__name__, identity.__doc__

('inner', None)

In [8]:
identity.__module__

'__main__'

In [9]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner
@trace
def identity(x):
    "I do nothing useful."
    return x

In [10]:
identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

In [ ]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
        functools.update_wrapper(inner, func)
    return inner

In [11]:
import functools
def trace(func):
     @functools.wraps(func)
     def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
     return inner
@trace
def identity(x):
    "I do nothing useful."
    return x

In [12]:
identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

Время выполнения функции

In [24]:
 import time
 def timethis(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)

    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float("inf")
        for i in range(n_iter):
            tick = time.time()
            result = func(*args, **kwargs)
            acc = min(acc, time.time() - tick)
        print(acc)
        return result
    return inner

 result = timethis(sum)(range(10 ** 6))

sum ... 0.13200592994689941


In [25]:
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)

    inner.ncalls = 0
    return inner

@profiled
def identity(x):
    return x

identity(42)

identity.ncalls

1

In [26]:
def once(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner

@once
def initialize_settings():
    print("Settings initialized.")

initialize_settings()
initialize_settings()

Settings initialized.


## Practice 

* Create a function getting random number of arguments.
* Find the number of digits in product of all arguments in the power of 1000
* You need to evaluate the time of work of this function

In [32]:
from time import time
import random

def number_of_digits(*args):
    res = 1
    for item in args:
        temp =  item ** 1000
        res *= temp
    return len(str(res))

def duration(f):
    def wrapper(*args):
        begin = time()
        res = f(*args)
        print(res)
        end = time()
        time_answer = end - begin
        print(time_answer)
    return wrapper
n = 100
arr = [random.randint(1000,3000) for item in range(n)]

number = duration(number_of_digits)
number(*arr)

327675
8.881756067276001


In [37]:
def duration(f):
    def wrapper(*args):
        begin = time()
        res = f(*args)
        print(res)
        end = time()
        time_answer = end - begin
        print(time_answer)
    return wrapper
@duration
def number_of_digits(*args):
    res = 1
    for item in args:
        temp =  item ** 1000
        res *= temp
    return len(str(res))
n = 100
arr = [random.randint(1000,3000) for item in range(n)]
number_of_digits(*arr)

329706
8.80516767501831


In [35]:
def number_of_digits(*args):
    res = 1
    for item in args:
        temp =  item ** 1000
        res *= temp
    return len(str(res))
def duration(f, *args):
    begin = time()
    res = number_of_digits(*args)
    print(res)
    end = time()
    time_answer = end - begin
    print(time_answer)
n = 100
arr = [random.randint(1000,3000) for item in range(n)]
duration(number_of_digits, *arr)

327675
8.487214803695679


## Lection

** iterators **

### Итерируемая последовательность (aka Iterable)

Это обьект у которого определён метод \_\_iter\_\_, возвращающий обьект реализующий протокол *итератора* 
(Примеры: list, dict, file, range)

### Итератор
Это обьект у которого определён метод \_\_next\_\_ (это может быть как отдельный обьект, так и, например, self самой последовательности, то есть она может быть итератором по самой себе)


Метод __next__ при каждом вызове должен возвращать следующий элемент последовательности, или выкидывать исключение  StopIteration, если последовательность кончилась

In [3]:
a = [1, 2, 3]
it = iter(a)
it

In [4]:
next(it)

1

In [6]:
lst = iter([1])
next(lst)
next(lst, 2)

2

In [5]:
def make_timer(ticks):
    def timer():
        nonlocal ticks
        ticks -= 1
        return ticks
    return timer

for t in iter(make_timer(10), 0):    # iter(function , terminal_value)
    print(t, end=' - ')

9 - 8 - 7 - 6 - 5 - 4 - 3 - 2 - 1 - 

In [8]:
seq = [1, 2, 3]
for x in seq:
    print(x)

1
2
3


In [ ]:
it = iter(seq)
while True:
    try:
        value = next(it)
        handle(value)
    except StopIteration:
        break

** generators **

Генератор это функция исполнение которой приостанавливается, а не прекращается при возврате значения. Выполнение функции можно продолжить с того же места.

In [38]:
def f():
    yield 1
    yield 2
    yield 3
for item in f():
    print(item)

1
2
3


In [9]:
def f(x):
    print('Generator enter')
    yield x
    x += 2
    yield x
    print('Generator Done')      

print('initial : type(f)', type(f))
a = f(5)
print('object created : type(a)', type(a))
print('first', next( a))
print('second', next(a))
print('2.5')
print('third', next(a))

initial : type(f) <class 'function'>
object created : type(a) <class 'generator'>
Generator enter
first 5
second 7
2.5
Generator Done


StopIteration: 

In [11]:
 def gen_AB(): #
    print('start')
    yield 'A' #
    print('continue')
    yield 'B' #
    print('end.') #
 for c in gen_AB(): #
    print('-->', c)

start
--> A
continue
--> B
end.


In [39]:
def fib():
    n0 = 0
    n1 = 1
    while True:
        yield n0
        n0, n1 = n1, n0+n1
        
for item in fib():
    print(item)
    if item > 10000:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946


In [40]:
def fib(b):
    n0 = 0
    n1 = 1
    while True:
        yield n0
        n0, n1 = n1, n0+n1
        if n0 > b:
            return
        
for item in fib(1000):
    print(item)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


In [41]:
def fib(b):
    n0 = 0
    n1 = 1
    while True:
        yield n0
        n0, n1 = n1, n0+n1
        if n0 > b:
            return

g = fib(1000)
it = g.__iter__()
print(it.__next__())
print(it.__next__())
print(it.__next__())
print(it.__next__())
print(it.__next__())
print(it.__next__())
print(it.__next__())
print(it.__next__())

0
1
1
2
3
5
8
13


In [47]:
def fib(b):
    n0 = 0
    n1 = 1
    while True:
        yield n0
        n0, n1 = n1, n0+n1
        if n0 > b:
            return

it = fib(1000)
#it = iter(g)
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

0
1
1
2
3


In [10]:
def unique(seq):
    seen = set()
    for elem in seq:
        if elem not in seen:
            seen.add(elem)
            yield elem

list(unique([1, 2, 3, 1, 2, 4]))

[1, 2, 3, 4]

In [11]:
def pmap(function, iterable):
    for i in iterable:
        yield function(i)

def pfilter(function, iterable):
    for i in iterable:
        if function(i):
            yield i

def pzip(*iterables):
    iters = list(pmap(iter, iterables))
    while True:
        try:
            yield [next(it) for it in iters]
        except StopIteration:
            return
list(pfilter(
    lambda x : not x[0] % x[1], 
    pzip(
        range(101, 200),
        range(2, 100))
    )
)

[[102, 3], [108, 9], [110, 11], [132, 33], [198, 99]]

** Lazy evaluation **

[to read](https://realpython.com/introduction-to-python-generators/#:~:text=When%20you%20call%20a%20generator,is%20executed%20up%20to%20yield%20.)

In [7]:
arr = [item for item in range(10000000)]
# arr

In [5]:
arr = (item for item in range(10000000))
arr

<generator object <genexpr> at 0x000002D73DE7E190>

In [12]:
 def gen_AB(): #
    print('start')
    yield 'A' #
    print('continue')
    yield 'B' #
    print('end.') #

In [13]:
res1 = [x*3 for x in gen_AB()]
res1

start
continue
end.


['AAA', 'BBB']

In [14]:
res2 = (x*3 for x in gen_AB())
for i in res2: 
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [15]:
range(1,100,1)

range(1, 100)

In [ ]:
arr = [2,6,8,4,5,7]
result = (item *2 for item in arr if item %2 == 1)
result

In [ ]:
arr = [2,6,8,4,5,7]
result = map(lambda n: n*2, arr)
print(result)

In [12]:
def aritprog_gen(begin=0, end=None, step=1):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index
for item in aritprog_gen(1,100,2):
    print(item)

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99


** itertools **

In [20]:
import itertools
gen = itertools.count(1, .5)
for item in range(10):
    print(next(gen))

1
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5


In [13]:
import itertools
gen = itertools.count(1, .5)
print(gen)
# for item in gen:
#     print(item)

count(1, 0.5)


In [ ]:
import itertools
gen = itertools.count(1, .5)
list(gen)

In [3]:
import itertools
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
print(list(gen))

[1, 1.5, 2.0, 2.5]


In [ ]:
import itertools
def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

** Examples **

** 1 **

count the number of rows in a CSV file

In [18]:
def csv_reader(file_name):
    file = open(file_name,"r", encoding = "utf-8")
    result = file.read().split("\n")
    return result

In [16]:
def csv_reader(file_name):
    for row in open(file_name, "r",encoding = "utf-8"):
        yield row

In [19]:
csv_gen = csv_reader("data\\file.csv")
row_count = 0

for row in csv_gen:
    row_count += 1

print(f"Row count is {row_count}")

Row count is 64462


In [ ]:
csv_gen = (row for row in open(file_name))

** Perfomance **

In [11]:
import sys
nums_squared_lc = [i * 2 for i in range(10000)]
print(sys.getsizeof(nums_squared_lc))
nums_squared_gc = (i ** 2 for i in range(10000))
print(sys.getsizeof(nums_squared_gc))

87616
112
